In [1]:
# Initial imports
import numpy as np
import pandas as pd
# import hvplot.pandas
from pathlib import Path
# import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

In [2]:
# import Final_project_NYC_BIKE_LANE_cleaned.csv data to match accident site 
file_path = "Resources/Segment2/Final_project_NYC_CRASH_WEATHER_Tableau.csv"
crash_weather_df = pd.read_csv(file_path, index_col=0 )
print(crash_weather_df.shape)
crash_weather_df

(9193, 25)


,COLLISION_ID,DATES,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,...,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,STREET,TEMP,VISIBILITY,HUMIDITY,WIND_SPEED,WEATHER_MAIN
0,4271563,2020-01-02,13:00:00,MANHATTAN,10003,40.714165,-74.006320,1.0,0,0,...,Driver Inattention/Distraction,Unspecified,sedan,Bike,CHAMBERS STREET,33.94,10000.0,61,9.17,Clear
1,4268322,2020-01-02,16:00:00,MANHATTAN,10012,40.725643,-73.992070,1.0,0,0,...,Turning Improperly,Unspecified,sedan,Bike,BOWERY,42.03,10000.0,54,9.17,Clear
2,4268207,2020-01-02,17:00:00,BROOKLYN,11221,40.693874,-73.917770,1.0,0,0,...,Traffic Control Disregarded,Unspecified,station wagon/sport utility vehicle,Bike,CENTRAL AVENUE,44.60,10000.0,48,12.75,Rain
3,4268408,2020-01-03,08:00:00,BRONX,10456,40.820747,-73.906006,1.0,0,0,...,Driver Inattention/Distraction,Unspecified,e-bike,Taxi,JACKSON AVENUE,46.78,10000.0,60,8.05,Rain
4,4269187,2020-01-03,11:00:00,BROOKLYN,11205,40.691017,-73.954475,1.0,0,0,...,View Obstructed/Limited,Unsafe Speed,sedan,E-Bike,DE KALB AVENUE,44.92,10000.0,74,8.05,Rain
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9188,4571618,2022-10-09,01:00:00,QUEENS,11375,40.734295,-73.850310,1.0,0,0,...,Traffic Control Disregarded,Unspecified,taxi,Bike,108 STREET,52.79,10000.0,53,8.05,Clear
9189,4571590,2022-10-09,06:00:00,BROOKLYN,11201,40.689840,-73.978630,1.0,0,0,...,Failure to Yield Right-of-Way,Unspecified,bike,Station Wagon/Sport Utility Vehicle,DE KALB AVENUE,48.74,10000.0,66,8.01,Clear
9190,4571393,2022-10-09,11:00:00,BROOKLYN,11201,40.702610,-73.992516,1.0,0,0,...,Driver Inattention/Distraction,Pedestrian/Bicyclist/Other Pedestrian Error/Co...,bike,Bike,FRONT STREET,44.96,10000.0,74,4.61,Clear
9191,4571698,2022-10-09,12:00:00,MANHATTAN,10002,40.715080,-73.992500,1.0,0,0,...,Turning Improperly,Unspecified,bike,Station Wagon/Sport Utility Vehicle,CANAL STREET,46.53,10000.0,72,5.75,Clear


In [3]:
# import Final_project_NYC_BIKE_LANE_cleaned.csv data to match accident site 
file_path = "Resources/Segment2/Final_project_NYC_BIKE_LANE_cleaned.csv"
NYC_Bike_lane = pd.read_csv(file_path, index_col=0 )
print(NYC_Bike_lane.shape)
NYC_Bike_lane.head(10)

(1292, 9)


,BOROUGH,STREET,BIKE_DIR,LANE_COUNT,FT_FACILIT,LATITUDE1,LONGITUDE1,LATITUDE2,LONGITUDE2
0,STATEN ISLAND,CONFERENCE HOUSE PARK GREENWAY,2,2,Greenway,40.500876,-74.249928,40.500563,-74.249690
10,STATEN ISLAND,HYLAN BLVD,2,2,Standard,40.508199,-74.227652,40.507753,-74.230064
13,STATEN ISLAND,MOUNT LORETTO GREENWAY,2,2,Greenway,40.505101,-74.221248,40.503424,-74.219395
28,MANHATTAN,79 ST NORTH ACCESS RAMP,2,2,Curbside,40.557564,-74.196975,40.556599,-74.196018
45,STATEN ISLAND,ARLENE ST,2,2,Standard,40.602796,-74.166948,40.602075,-74.167196
46,STATEN ISLAND,STEINWAY AVE,2,2,Sharrows,40.590455,-74.164487,40.590139,-74.164829
51,STATEN ISLAND,TRAVIS AVE,2,2,Standard,40.591808,-74.160136,40.592240,-74.160895
54,STATEN ISLAND,GULF AV,2,2,Sidewalk,40.629973,-74.186760,40.629947,-74.186767
66,STATEN ISLAND,GUYON AV,2,2,Sharrows,40.564444,-74.126753,40.563975,-74.125980
71,STATEN ISLAND,S RAILROAD AV,2,2,Sharrows,40.565889,-74.125142,40.565326,-74.125709


In [4]:
print(len(crash_weather_df['STREET'].unique()))
a = crash_weather_df['STREET'].unique()
print(len(NYC_Bike_lane['STREET'].unique()))
b = NYC_Bike_lane['STREET'].unique()

1673
1292


In [5]:
len(set(a) & set(b))

57

In [6]:
# getting list of lat and lon to pass to have range of lat1 -lat2 and lon1-lon2
lat_list = []
lon_list = []
for i in range(len(NYC_Bike_lane['LATITUDE1'])):
    try:
        lat_range = NYC_Bike_lane['LATITUDE2'][i], NYC_Bike_lane['LATITUDE1'][i]
        lon_range = NYC_Bike_lane['LONGITUDE1'][i], NYC_Bike_lane['LONGITUDE2'][i]
        lat_list.append(lat_range)
        lon_list.append(lon_range)
    except:
        pass

In [7]:
def Check_Coordinates(lat, lon):
    if not any([start < lat < end for start, end in lat_list]) \
        and not any([start < lon < end for start, end in lon_list]):
        return 0
    else:
        return 1

In [8]:
# Testing function
#crash_weather_df['LAT_FILTERED'] = crash_weather_df['LATITUDE'].apply(lambda x: 0 if not any([a < x < b for a,b in lat_list]) else 1)

In [9]:
# Final version
crash_weather_df['BIKE_LANE'] = crash_weather_df.apply(lambda x: Check_Coordinates(x['LATITUDE'], x['LONGITUDE']), axis=1)

In [14]:
crash_weather_df.head(5)

,COLLISION_ID,DATES,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,...,CONTRIBUTING FACTOR VEHICLE 2,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,STREET,TEMP,VISIBILITY,HUMIDITY,WIND_SPEED,WEATHER_MAIN,BIKE_LANE
0,4271563,2020-01-02,13:00:00,MANHATTAN,10003,40.714165,-74.006320,1.0,0,0,...,Unspecified,sedan,Bike,CHAMBERS STREET,33.94,10000.0,61,9.17,Clear,1
1,4268322,2020-01-02,16:00:00,MANHATTAN,10012,40.725643,-73.992070,1.0,0,0,...,Unspecified,sedan,Bike,BOWERY,42.03,10000.0,54,9.17,Clear,1
2,4268207,2020-01-02,17:00:00,BROOKLYN,11221,40.693874,-73.917770,1.0,0,0,...,Unspecified,station wagon/sport utility vehicle,Bike,CENTRAL AVENUE,44.60,10000.0,48,12.75,Rain,0
3,4268408,2020-01-03,08:00:00,BRONX,10456,40.820747,-73.906006,1.0,0,0,...,Unspecified,e-bike,Taxi,JACKSON AVENUE,46.78,10000.0,60,8.05,Rain,0
4,4269187,2020-01-03,11:00:00,BROOKLYN,11205,40.691017,-73.954475,1.0,0,0,...,Unsafe Speed,sedan,E-Bike,DE KALB AVENUE,44.92,10000.0,74,8.05,Rain,1


In [16]:
crash_weather_df.columns.tolist()

['COLLISION_ID',
 'DATES',
 'TIME',
 'BOROUGH',
 'ZIP CODE',
 'LATITUDE',
 'LONGITUDE',
 'NUMBER OF PERSONS INJURED',
 'NUMBER OF PERSONS KILLED',
 'NUMBER OF PEDESTRIANS INJURED',
 'NUMBER OF PEDESTRIANS KILLED',
 'NUMBER OF CYCLIST INJURED',
 'NUMBER OF CYCLIST KILLED',
 'NUMBER OF MOTORIST INJURED',
 'NUMBER OF MOTORIST KILLED',
 'CONTRIBUTING FACTOR VEHICLE 1',
 'CONTRIBUTING FACTOR VEHICLE 2',
 'VEHICLE TYPE CODE 1',
 'VEHICLE TYPE CODE 2',
 'STREET',
 'TEMP',
 'VISIBILITY',
 'HUMIDITY',
 'WIND_SPEED',
 'WEATHER_MAIN',
 'BIKE_LANE']

In [12]:
crash_weather_df.groupby(["BIKE_LANE"]).count()

,COLLISION_ID,DATES,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,...,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,STREET,TEMP,VISIBILITY,HUMIDITY,WIND_SPEED,WEATHER_MAIN
BIKE_LANE,,,,,,,,,,,,,,,,,,,,,
0,5998,5998,5998,5998,5998,5998,5998,5998,5998,5998,...,5998,5998,5998,5998,5998,5998,5990,5998,5998,5998
1,3195,3195,3195,3194,3195,3195,3195,3195,3195,3195,...,3195,3195,3195,3195,3195,3195,3190,3195,3195,3195


In [13]:
# crash_weather_df.to_csv("NYC_Bike_Crash_Weather_Bikelane_All.csv")

In [ ]:
NYC["DATES"]= NYC['DATES'].apply(lambda x: x.split(',',1)[1])

In [ ]:
NYC["MONTH"]=NYC['DATES'].apply(lambda x: x.strip().split(' ')[0])

In [ ]:
NYC.head(5)

,COLLISION_ID,DATES,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,...,VEHICLE TYPE CODE 2,STREET,TEMP,VISIBILITY,HUMIDITY,WIND_SPEED,WEATHER_MAIN,BIKE_LANE,DAY_OF_WEEK,MONTH
0,4271563,"January 2, 2020",13:00:00,MANHATTAN,10003,40.714165,-74.006320,1,0,0,...,Bike,CHAMBERS STREET,33.94,10000.0,61,9.17,Clear,1,Thursday,January
1,4268322,"January 2, 2020",16:00:00,MANHATTAN,10012,40.725643,-73.992070,1,0,0,...,Bike,BOWERY,42.03,10000.0,54,9.17,Clear,1,Thursday,January
2,4268207,"January 2, 2020",17:00:00,BROOKLYN,11221,40.693874,-73.917770,1,0,0,...,Bike,CENTRAL AVENUE,44.60,10000.0,48,12.75,Rain,0,Thursday,January
3,4268408,"January 3, 2020",8:00:00,BRONX,10456,40.820747,-73.906006,1,0,0,...,Taxi,JACKSON AVENUE,46.78,10000.0,60,8.05,Rain,0,Friday,January
4,4269187,"January 3, 2020",11:00:00,BROOKLYN,11205,40.691017,-73.954475,1,0,0,...,E-Bike,DE KALB AVENUE,44.92,10000.0,74,8.05,Rain,1,Friday,January


In [ ]:
NYC.columns.tolist()

['COLLISION_ID',
 'DATES',
 'TIME',
 'BOROUGH',
 'ZIP CODE',
 'LATITUDE',
 'LONGITUDE',
 'NUMBER OF PERSONS INJURED',
 'NUMBER OF PERSONS KILLED',
 'NUMBER OF PEDESTRIANS INJURED',
 'NUMBER OF PEDESTRIANS KILLED',
 'NUMBER OF CYCLIST INJURED',
 'NUMBER OF CYCLIST KILLED',
 'NUMBER OF MOTORIST INJURED',
 'NUMBER OF MOTORIST KILLED',
 'CONTRIBUTING FACTOR VEHICLE 1',
 'CONTRIBUTING FACTOR VEHICLE 2',
 'VEHICLE TYPE CODE 1',
 'VEHICLE TYPE CODE 2',
 'STREET',
 'TEMP',
 'VISIBILITY',
 'HUMIDITY',
 'WIND_SPEED',
 'WEATHER_MAIN',
 'BIKE_LANE',
 'DAY_OF_WEEK',
 'MONTH']

In [ ]:
NYC = NYC[['COLLISION_ID','DATES','TIME','DAY_OF_WEEK', 'MONTH',
 'BOROUGH',
 'ZIP CODE',
 'LATITUDE',
 'LONGITUDE',
 'NUMBER OF PERSONS INJURED',
 'NUMBER OF PERSONS KILLED',
 'NUMBER OF PEDESTRIANS INJURED',
 'NUMBER OF PEDESTRIANS KILLED',
 'NUMBER OF CYCLIST INJURED',
 'NUMBER OF CYCLIST KILLED',
 'NUMBER OF MOTORIST INJURED',
 'NUMBER OF MOTORIST KILLED',
 'CONTRIBUTING FACTOR VEHICLE 1',
 'CONTRIBUTING FACTOR VEHICLE 2',
 'VEHICLE TYPE CODE 1',
 'VEHICLE TYPE CODE 2',
 'STREET',
 'TEMP',
 'VISIBILITY',
 'HUMIDITY',
 'WIND_SPEED',
 'WEATHER_MAIN',
 'BIKE_LANE']]

In [ ]:
# NYC.to_csv("NYC_Bike_All_xTableau.csv")